# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import struct
import time
from decimal import Decimal

# Get weight values

In [2]:
total_num_of_weight = 401
def get_weights_arr():
    weights_arr = []
    w_val = Decimal('-2.0')
    step = Decimal('0.01')
    for i in range (total_num_of_weight):
        weights_arr.append(float(w_val))
        w_val += step
    return weights_arr

weights_arr = get_weights_arr()
#the weight value to be recovered
true_weight = 1.43

# Function Definitions

In [3]:
def float_to_binary_str(f):
    # Pack the float into 4 bytes (32-bit) using IEEE 754 standard
    [packed] = struct.unpack('!I', struct.pack('!f', f))
    # Convert the packed number to a binary string
    return f"{packed:032b}"

In [4]:
def HW_float32(f):
    # Get the binary representation of the 32-bit float
    binary_str = float_to_binary_str(f)
    # Count and return the number of '1' bits
    return binary_str.count('1')

In [5]:
#get one part of the binary representation
def getbyte(f,byte_position):
    if byte_position == 0:#sign bit
        inbinary = float_to_binary_str(f)[0]
    elif byte_position == 4:#last 7 bits
        inbinary = float_to_binary_str(f)[25:32]
    else:
        inbinary = float_to_binary_str(f)[(byte_position-1)*8+1:byte_position*8+1]
    
    return int(inbinary,2)

#### The values to be recovered

In [6]:
#true values of different parts of the weight
true_sign_bit = getbyte(true_weight,0)
true_exponent_bits = getbyte(true_weight,1)
true_byte_two = getbyte(true_weight,2)
true_byte_three = getbyte(true_weight,3)
true_byte_four = getbyte(true_weight,4)
true_value_index = weights_arr.index(true_weight)

In [7]:
print("sign bit = " + str(true_sign_bit) + ", exponent = " + str(true_exponent_bits) + ", first mantissa byte = " + str(true_byte_two) + ", second mantissa byte = " + str(true_byte_three) + ", last 7 bits = " + str(true_byte_four))

sign bit = 0, exponent = 127, first mantissa byte = 110, second mantissa byte = 20, last 7 bits = 61


In [8]:
def get_hypothetical_leakages(num_of_traces, inputs_arr):
    
    # Compute all hypothetical products at once using broadcasting
    hypothetical_products = np.outer(weights_arr, inputs_arr[:num_of_traces])  # Shape: (total_num_of_weight, num_of_traces)

    # Apply the HW function element-wise using NumPy's vectorization
    hypothetical_leakages = np.vectorize(HW_float32)(hypothetical_products)  
    return np.array(hypothetical_leakages)

In [9]:
def recover_byte(correlation_arr, byte_position, total_time_sample):
    num_of_different_values = 256 #for byte two and three and exponent bits, we have 256 different values
    if byte_position == 0:#for sign bit, there are just 2 different values
        num_of_different_values = 2
    if byte_position == 4:#for last 7 bits, there are 2^7 different values
        num_of_different_values = 128
    correlations = [[0 for t in range(total_time_sample)] for i in range(num_of_different_values)]
    for weight_index in range(total_num_of_weight):#for each weight
        byte_value = getbyte(weights_arr[weight_index],byte_position)#for the chosen byte of this weight value
        for t in range(total_time_sample):#take the correlations for this weight value, for each time sample, update the correlations
            if correlation_arr[weight_index][t] > correlations[byte_value][t]:
                correlations[byte_value][t] = correlation_arr[weight_index][t]
        
    return correlations

In [10]:
#### save the plot raw data
def saveplot(foldername, filename, plot_data, arr_num_traces):                
    #save the file
    f = open(foldername + "//" + filename + ".txt", "w")    
    length = len(arr_num_traces)
    
    #first line---
    f.write("x y\n")
    #other lines
    for i in range(length):#for each line
        f.write(str(arr_num_traces[i])+" " + str(plot_data[i])+"\n")
    f.close()
    return

In [11]:
def find_max_and_max_true(correlation_arr, true_value_index):
    overall_max = 0
    true_max = 0
    total_plots = len(correlation_arr)
    
    for i in range(total_plots):
        if i == true_value_index:
            true_max = max(correlation_arr[i])
        else:
            w_max = max(correlation_arr[i])
            if w_max > overall_max:
                overall_max = w_max
                
    return overall_max, true_max

# Attack on protected traces

In [12]:
folder_name="protected"
num_of_traces = 50000
start = 490
end_pro = 4301
step_size = 500
total_pro = end_pro - start
print(total_pro)

3811


In [13]:
#protected case
num_trace_arr = []
for n in range(5,95,5):
    num_trace_arr.append(n)


for n in range(100, num_of_traces, step_size):
    num_trace_arr.append(n)
num_trace_arr.append(num_of_traces)
len(num_trace_arr)

119

In [14]:
def get_inputs(folder_name):
    with open(folder_name + '/inputs.txt') as f:
        inputs_arr = f.read().splitlines()
    f.close()
    inputs_arr = np.array(inputs_arr)
    inputs_arr = inputs_arr.astype(float)
    return inputs_arr

In [15]:
inputs_pro = get_inputs(folder_name)

In [16]:
hypothetical_leakages = get_hypothetical_leakages(num_of_traces=num_of_traces, inputs_arr=inputs_pro)

In [17]:
def CPA_protected(num_trace_arr, hypothetical_leakages, batch_size=10):
    total_experiment = len(num_trace_arr)
    max_traces = max(num_trace_arr)  

    r_all = np.zeros((total_experiment, total_num_of_weight, total_pro))

    print("Loading traces in batches...")

    def read_time_samples(trace_idx, start_idx, batch_size):
        with open(f"{folder_name}/trace_{trace_idx}.txt") as f:
            lines = f.readlines()[start_idx:start_idx + batch_size]
            return np.array([float(line.strip()) for line in lines])

    for batch_start in range(start, start + total_pro, batch_size):
        batch_end = min(batch_start + batch_size, start + total_pro)
        current_batch_size = batch_end - batch_start

        print(f"Processing time samples {batch_start} to {batch_end}...")
        
        start_time = time.time()

        traces = np.array([read_time_samples(i, batch_start, current_batch_size) for i in range(max_traces)])

        print("Processing trace subsets...")
        trace_subsets = np.full((total_experiment, max_traces, current_batch_size), np.nan)

        for index, trace_no in enumerate(num_trace_arr):
            trace_subsets[index, :trace_no, :current_batch_size] = traces[:trace_no, :current_batch_size].copy()

        print("Processing hypothetical leakages...")
        hypoleak_expanded = np.full((total_experiment, total_num_of_weight, max_traces), np.nan)

        for index, trace_no in enumerate(num_trace_arr):
            hypoleak_expanded[index, :, :trace_no] = hypothetical_leakages[:, :trace_no].copy()

        print("Computing correlations using np.corrcoef...")

        

        trace_slices = [trace_subsets[i, :num_trace_arr[i], :] for i in range(total_experiment)]
        hypoleak_slices = [hypoleak_expanded[i, :, :num_trace_arr[i]] for i in range(total_experiment)]

        corr_matrices = []
        for hypo, trace in zip(hypoleak_slices, trace_slices):
            combined_matrix = np.vstack([hypo, trace.T])  
            corr_matrices.append(np.corrcoef(combined_matrix))

        for i in range(total_experiment):
            corr_matrix = corr_matrices[i]

            # Extract the correct correlation values (hypothetical leakages vs. traces in the batch)
            extracted_corr = np.abs(corr_matrix[:total_num_of_weight, total_num_of_weight:total_num_of_weight + current_batch_size])
            
            # # Ensure shape consistency for the last batch
            # extracted_corr = extracted_corr[:, :current_batch_size]
            

            r_all[i, :, batch_start-start:batch_end-start] = extracted_corr  

        end_time = time.time()
        print(f"Time taken for batch {batch_start}-{batch_end}: {end_time - start_time:.2f} seconds")

    return r_all


In [18]:
def run_CPA_n_traces_protected(batch_size, total_ts, start_ts, save_folder, filename, arr_num_traces, hypothetical_leakages):
    # initialize np arrays for highest correlation results
    arr_max_sign_bit = np.array([])
    arr_max_true_sign_bit = np.array([])
    arr_max_exponent_bits = np.array([])
    arr_max_true_exponent_bits = np.array([])
    arr_max_byte_two = np.array([])
    arr_max_true_byte_two = np.array([])
    arr_max_byte_three = np.array([])
    arr_max_true_byte_three = np.array([])
    arr_max_byte_four = np.array([])
    arr_max_true_byte_four = np.array([])

    total_experiment = len(arr_num_traces)
    r_all = CPA_protected(num_trace_arr=arr_num_traces, hypothetical_leakages=hypothetical_leakages, batch_size=batch_size)

    for i in range(total_experiment):        
        
        if i%10 == 0:
            print("running for experiment number " + str(i))
        r = r_all[i]

        correlations_sign_bit = recover_byte(r, 0, total_ts)
        correlations_exponent_bits = recover_byte(r,1, total_ts)
        correlations_byte_two = recover_byte(r, 2, total_ts)
        correlations_byte_three = recover_byte(r, 3, total_ts)
        correlations_byte_four = recover_byte(r, 4, total_ts)

        max_sign_bit, max_true_sign_bit = find_max_and_max_true(correlations_sign_bit, true_sign_bit)
        max_exponent_bits, max_true_exponent_bits = find_max_and_max_true(correlations_exponent_bits, true_exponent_bits)
        max_byte_two, max_true_byte_two = find_max_and_max_true(correlations_byte_two, true_byte_two)
        max_byte_three, max_true_byte_three = find_max_and_max_true(correlations_byte_three, true_byte_three)
        max_byte_four, max_true_byte_four = find_max_and_max_true(correlations_byte_four, true_byte_four)

        arr_max_sign_bit = np.append(arr_max_sign_bit, max_sign_bit)
        arr_max_true_sign_bit = np.append(arr_max_true_sign_bit, max_true_sign_bit)
        arr_max_exponent_bits = np.append(arr_max_exponent_bits, max_exponent_bits)
        arr_max_true_exponent_bits = np.append(arr_max_true_exponent_bits, max_true_exponent_bits)
        arr_max_byte_two = np.append(arr_max_byte_two, max_byte_two)
        arr_max_true_byte_two = np.append(arr_max_true_byte_two, max_true_byte_two)
        arr_max_byte_three = np.append(arr_max_byte_three, max_byte_three)
        arr_max_true_byte_three = np.append(arr_max_true_byte_three, max_true_byte_three)
        arr_max_byte_four = np.append(arr_max_byte_four, max_byte_four)
        arr_max_true_byte_four = np.append(arr_max_true_byte_four, max_true_byte_four)

        if i%10 == 0:
            print("saving files")
            saveplot(save_folder, filename + "sign_bit", arr_max_sign_bit, arr_num_traces[:i])
            saveplot(save_folder, filename + "true_sign_bit", arr_max_true_sign_bit, arr_num_traces[:i])
            saveplot(save_folder, filename + "exponent_bits", arr_max_exponent_bits, arr_num_traces[:i])
            saveplot(save_folder, filename + "true_exponent_bits", arr_max_true_exponent_bits, arr_num_traces[:i])
            saveplot(save_folder, filename + "first_byte", arr_max_byte_two, arr_num_traces[:i])
            saveplot(save_folder, filename + "true_first_byte", arr_max_true_byte_two, arr_num_traces[:i])
            saveplot(save_folder, filename + "second_byte", arr_max_byte_three, arr_num_traces[:i])
            saveplot(save_folder, filename + "true_second_byte", arr_max_true_byte_three, arr_num_traces[:i])
            saveplot(save_folder, filename + "last_bits", arr_max_byte_four, arr_num_traces[:i])
            saveplot(save_folder, filename + "true_last_bits", arr_max_true_byte_four, arr_num_traces[:i])

            
    saveplot(save_folder, filename + "sign_bit", arr_max_sign_bit, arr_num_traces)
    saveplot(save_folder, filename + "true_sign_bit", arr_max_true_sign_bit, arr_num_traces)
    saveplot(save_folder, filename + "exponent_bits", arr_max_exponent_bits, arr_num_traces)
    saveplot(save_folder, filename + "true_exponent_bits", arr_max_true_exponent_bits, arr_num_traces)
    saveplot(save_folder, filename + "first_byte", arr_max_byte_two, arr_num_traces)
    saveplot(save_folder, filename + "true_first_byte", arr_max_true_byte_two, arr_num_traces)
    saveplot(save_folder, filename + "second_byte", arr_max_byte_three, arr_num_traces)
    saveplot(save_folder, filename + "true_second_byte", arr_max_true_byte_three, arr_num_traces)
    saveplot(save_folder, filename + "last_bits", arr_max_byte_four, arr_num_traces)
    saveplot(save_folder, filename + "true_last_bits", arr_max_true_byte_four, arr_num_traces)
    
    return

#### Results

In [19]:
run_CPA_n_traces_protected(batch_size = 3, total_ts=total_pro, start_ts=start, save_folder="plots_protected", filename="pro_", arr_num_traces=num_trace_arr, hypothetical_leakages=hypothetical_leakages)

Loading traces in batches...
Processing time samples 490 to 493...
Processing trace subsets...
Processing hypothetical leakages...
Computing correlations using np.corrcoef...
Time taken for batch 490-493: 84.69 seconds
Processing time samples 493 to 496...
Processing trace subsets...
Processing hypothetical leakages...
Computing correlations using np.corrcoef...
Time taken for batch 493-496: 73.51 seconds
Processing time samples 496 to 499...
Processing trace subsets...
Processing hypothetical leakages...
Computing correlations using np.corrcoef...
Time taken for batch 496-499: 74.15 seconds
Processing time samples 499 to 502...
Processing trace subsets...
Processing hypothetical leakages...
Computing correlations using np.corrcoef...
Time taken for batch 499-502: 74.28 seconds
Processing time samples 502 to 505...
Processing trace subsets...
Processing hypothetical leakages...
Computing correlations using np.corrcoef...
Time taken for batch 502-505: 73.96 seconds
Processing time sampl

## Bound calculations

In [62]:
neuron_input = 7
neuron_output = 5
zalpha = 4.5
l = neuron_input*neuron_output
c = 8*zalpha*zalpha

$$
\operatorname{bound} \approx\frac{110\ell^s}{\left(\ln\frac{1+\rho_1}{1-\rho_1}-\ln\frac{1+\rho_2}{1-\rho_2}\right)^2}
$$

where $\ell$ is the product of the number of input neurons and the number of output neurons of the target layer, $s=0$ for unprotected implementations and $s=2$ for the shuffling case.


In [63]:
def bound(shuffle, rho1, rho2):
    x = np.log((1+rho1)/(1-rho1))
    y = np.log((1+rho2)/(1-rho2))
    z = (x-y)*(x-y)
    w =(1/z)*c
    if shuffle == 1:
        w = w * l * l
    print("number of traces = " + str(w))
    #time per measurement of 450ms 
    #(including data transfer to the DUT, as well as data transfer from oscilloscope to computer and storing the data).
    t = (w*450)/(1000*60*60*24)#1ms to second, to minute, to hour, to day
    print('time = ' + str(t) + " days")
    return

In [71]:
def boud_DATE(shuffle, rho):
    w = 28/(rho*rho)
    if shuffle == 1:
        w = w * l * l
    print("number of traces = " + str(w) + "=" +str(w/1000000) + " million")
    #time per measurement of 450ms 
    #(including data transfer to the DUT, as well as data transfer from oscilloscope to computer and storing the data).
    t = (w*450)/(1000*60*60*24)#1ms to second, to minute, to hour, to day
    print('time = ' + str(t) + " days")
    return

In [72]:
def read_last_row_last_column(file_path, delimiter=None):
    """
    Reads the last row and last column from a given file.

    :param file_path: Path to the file.
    :param delimiter: Delimiter used in the file (e.g., ',' for CSV, None for whitespace).
    :return: The value in the last row and last column.
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()  # Read all lines

    if not lines:
        raise ValueError("File is empty.")

    last_row = lines[-1].strip()  # Get last row, remove whitespace

    if delimiter:
        columns = last_row.split(delimiter)  # Split based on the given delimiter
    else:
        columns = last_row.split()  # Split based on whitespace

    if not columns:
        raise ValueError("Last row is empty.")

    return float(columns[-1])  # Return the last column value

### Unprotected case

In [78]:
###exponent_bits
rho1 = read_last_row_last_column("plots_unprotected//exponent_bits.txt")
rho2 = read_last_row_last_column("plots_unprotected//true_exponent_bits.txt")
bound(0, rho1, rho2)
print("Bound from DATE")
boud_DATE(0,rho2)

number of traces = 59.42515104905821
time = 0.0003095059950471782 days
Bound from DATE
number of traces = 31.888063070671116=3.188806307067112e-05 million
time = 0.00016608366182641206 days


### Protected case

In [74]:
###exponent_bits
rho1 = read_last_row_last_column("plots_protected//pro_exponent_bits.txt")
rho2 = read_last_row_last_column("plots_protected//pro_true_exponent_bits.txt")
bound(1, rho1, rho2)
print("Bound from DATE")
boud_DATE(1,rho2)

number of traces = 786124.6614373528
time = 4.094399278319545 days
Bound from DATE
number of traces = 1318181.2901658402=1.3181812901658403 million
time = 6.865527552947085 days


In [75]:
###first byte
rho1 = read_last_row_last_column("plots_protected//pro_first_byte.txt")
rho2 = read_last_row_last_column("plots_protected//pro_true_first_byte.txt")
bound(1, rho1, rho2)
print("Bound from DATE")
boud_DATE(1,rho2)

number of traces = 734274.1592475261
time = 3.8243445794141984 days
Bound from DATE
number of traces = 1468972.772434347=1.468972772434347 million
time = 7.650899856428891 days


In [76]:
###second byte
rho1 = read_last_row_last_column("plots_protected//pro_second_byte.txt")
rho2 = read_last_row_last_column("plots_protected//pro_true_second_byte.txt")
bound(1, rho1, rho2)
print("Bound from DATE")
boud_DATE(1,rho2)

number of traces = 734274.1592475261
time = 3.8243445794141984 days
Bound from DATE
number of traces = 1468972.772434347=1.468972772434347 million
time = 7.650899856428891 days


In [77]:
###last bits
rho1 = read_last_row_last_column("plots_protected//pro_last_bits.txt")
rho2 = read_last_row_last_column("plots_protected//pro_true_last_bits.txt")
bound(1, rho1, rho2)
print("Bound from DATE")
boud_DATE(1,rho2)

number of traces = 734274.1592475261
time = 3.8243445794141984 days
Bound from DATE
number of traces = 1468972.772434347=1.468972772434347 million
time = 7.650899856428891 days


## Other codes

In [20]:
# import shutil
# shutil.make_archive("plots", 'zip', "plots")

In [21]:
##THIS ONE TRIES TO VECTORIZE EVERYTHING NO LOOPS, COMPUTING CORRELATIONS WITH PYTHON FUNCTION
#CRASHES WITH 2000 TRACES ON MAC
#def CPA_protected(num_trace_arr, hypothetical_leakages):
#     total_experiment = len(num_trace_arr)
#     max_traces = max(num_trace_arr)  # Maximum number of traces across all experiments

#     # Initialize r_all using NumPy
#     r_all = np.zeros((total_experiment, total_num_of_weight, total_pro))

#     print("Loading traces...")
#     # Efficiently read all required time samples for all traces (Shape: (max_traces, total_pro))
#     def read_time_samples(i):
#         with open(f"{folder_name}/trace_{i}.txt") as f:
#             return np.array([float(line.strip()) for line in f.readlines()[start:start + total_pro]])

#     traces = np.array([read_time_samples(i) for i in range(max_traces)])  # Shape: (max_traces, total_pro)

#     print("Real leakages computations...")
#     # **Preallocate expanded trace subsets array**
#     trace_subsets = np.full((total_experiment, max_traces, total_pro), np.nan)  # Fill with NaNs

#     # Fill trace subsets dynamically
#     for index, trace_no in enumerate(num_trace_arr):
#         trace_subsets[index, :trace_no, :] = traces[:trace_no, :]

#     print("Hypothetical leakages computations...")
#     # **Preallocate and fill hypothetical leakages**
#     hypoleak_expanded = np.full((total_experiment, total_num_of_weight, max_traces), np.nan)  # Fill with NaNs
#     for index, trace_no in enumerate(num_trace_arr):
#         hypoleak_expanded[index, :, :trace_no] = hypothetical_leakages[:, :trace_no]

#     print("Preparing for correlation computations...")
#     # Expand hypoleak_expanded along last axis to match trace_subsets (total_pro)
#     hypoleak_expanded = np.repeat(hypoleak_expanded[:, :, :, None], total_pro, axis=-1)  # Shape: (total_experiment, total_num_of_weight, max_traces, total_pro)

#     # Ensure trace_subsets is correctly shaped for concatenation
#     trace_subsets = trace_subsets[:, None, :, :]  # Shape: (total_experiment, 1, max_traces, total_pro)

#     # Concatenate hypothetical leakages and traces along axis=1
#     combined_matrix = np.concatenate([hypoleak_expanded, trace_subsets], axis=1)  # Shape: (total_experiment, total_num_of_weight+1, max_traces, total_pro)

#     print("Computing correlations in parallel...")
#     # Compute correlation matrix for all time samples at once (fully vectorized)
#     corr_matrix = np.array([[np.corrcoef(combined_matrix[exp, :, :, t])[0:-1, -1] for t in range(total_pro)] for exp in range(total_experiment)])

#     # Store absolute correlation values
#     r_all = np.abs(corr_matrix).transpose(0, 2, 1)  # Shape: (total_experiment, total_num_of_weight, total_pro)

#     return r_all


In [22]:
#THIS FUNCTION LOADS ALL TRACE DATA, BUT CRASHES ON BOTH COMPUTERS
# def CPA_protected(num_trace_arr, hypothetical_leakages):
#     total_experiment = len(num_trace_arr)
#     max_traces = max(num_trace_arr)  # Maximum number of traces across all experiments

#     # Initialize r_all using NumPy
#     r_all = np.zeros((total_experiment, total_num_of_weight, total_pro))

#     print("Loading traces...")
#     # Efficiently read all required time samples for all traces (Shape: (max_traces, total_pro))
#     def read_time_samples(i):
#         with open(f"{folder_name}/trace_{i}.txt") as f:
#             return np.array([float(line.strip()) for line in f.readlines()[start:start + total_pro]])

#     traces = np.array([read_time_samples(i) for i in range(max_traces)])  # Shape: (max_traces, total_pro)

#     print("Real leakages computations...")
#     trace_subsets = np.full((total_experiment, max_traces, total_pro), np.nan)  # Fill with NaNs

#     # Fill trace subsets dynamically
#     for index, trace_no in enumerate(num_trace_arr):
#         trace_subsets[index, :trace_no, :] = traces[:trace_no, :]

#     print("Hypothetical leakages computations...")
#     hypoleak_expanded = np.full((total_experiment, total_num_of_weight, max_traces), np.nan)

#     for index, trace_no in enumerate(num_trace_arr):
#         hypoleak_expanded[index, :, :trace_no] = hypothetical_leakages[:, :trace_no]

#     print("Computing correlations using np.corrcoef...")

#     start_time = time.time()  # Start timing

#     for i in range(total_pro):
#         if i % 100 == 0 and i > 0:  # Log the time taken every 10 time samples
#             elapsed_time = time.time() - start_time
#             print(f"Time taken for {i} time samples: {elapsed_time:.2f} seconds")
#             start_time = time.time()  # Reset the timer for the next batch

#         # Extract traces at current time sample
#         traces_at_time_sample = trace_subsets[:, :, i]  # Shape: (total_experiment, max_traces)

#         # Compute correlation for all experiments in parallel
#         for index in range(total_experiment):
#             trace_no = num_trace_arr[index]
#             trace_slice = traces_at_time_sample[index, :trace_no]  # Shape: (trace_no,)
#             hypoleak_slice = hypoleak_expanded[index, :, :trace_no]  # Shape: (total_num_of_weight, trace_no)

#             # Stack traces and hypothetical leakages together for correlation computation
#             combined_matrix = np.vstack([hypoleak_slice, trace_slice])  # Shape: (total_num_of_weight+1, trace_no)

#             # Compute correlation matrix
#             corr_matrix = np.corrcoef(combined_matrix)  # Shape: (total_num_of_weight+1, total_num_of_weight+1)

#             # Extract correlation values (last row, except last element)
#             r_all[index, :, i] = np.abs(corr_matrix[:-1, -1])  # Extract absolute correlation

#     return r_all

In [23]:
#THIS ONE TRIES TO VECTORIZE EVERYTHING NO LOOPS, COMPUTING CORRELATIONS FROM SCRATCH
#CRASHES WITH 2000 TRACES ON MAC
#def CPA_protected(num_trace_arr, hypothetical_leakages):
#     total_experiment = len(num_trace_arr)
#     max_traces = max(num_trace_arr)  # Maximum number of traces across all experiments

#     # Initialize r_all using NumPy
#     r_all = np.zeros((total_experiment, total_num_of_weight, total_pro))
    
#     print("Loading traces...")
#     # Efficiently read all required time samples for all traces (Shape: (max_traces, total_pro))
#     def read_time_samples(i):
#         with open(f"{folder_name}/trace_{i}.txt") as f:
#             return np.array([float(line.strip()) for line in f.readlines()[start:start + total_pro]])

#     traces = np.array([read_time_samples(i) for i in range(max_traces)])  # Shape: (max_traces, total_pro)

#     print("Real leakages computations...")
#     # **Fix: Preallocate the expanded trace subsets array**
#     trace_subsets = np.full((total_experiment, max_traces, total_pro), np.nan)  # Fill with NaNs

#     # Fill trace subsets dynamically
#     for index, trace_no in enumerate(num_trace_arr):
#         trace_subsets[index, :trace_no, :] = traces[:trace_no, :]

#     # Compute mean and variance for traces (Shape: (total_experiment, 1, total_pro))
#     mean_traces = np.nanmean(trace_subsets, axis=1, keepdims=True)  # Ignore NaNs
#     trace_dev = trace_subsets - mean_traces  # Shape: (total_experiment, max_traces, total_pro)
#     var_traces = np.nansum(trace_dev**2, axis=1, keepdims=True)  # Shape: (total_experiment, 1, total_pro)
    
#     print("Hypothetical leakages computations...")
#     # **Fix: Preallocate and fill hypothetical leakages**
#     hypoleak_expanded = np.full((total_experiment, total_num_of_weight, max_traces), np.nan)  # Fill with NaNs
#     for index, trace_no in enumerate(num_trace_arr):
#         hypoleak_expanded[index, :, :trace_no] = hypothetical_leakages[:, :trace_no]

#     # Precompute means and variances for hypothetical leakages (Shape: (total_experiment, total_num_of_weight, 1))
#     precomputed_means = np.nanmean(hypoleak_expanded, axis=2, keepdims=True)  # Ignore NaNs
#     precomputed_variances = np.nansum((hypoleak_expanded - precomputed_means) ** 2, axis=2, keepdims=True)

#     print("Correlations computations...")
#     # Compute correlation for all time samples at once (fully vectorized)
#     # **Fix: Reshape trace_dev to match the expected broadcasting dimensions**
#     trace_dev = trace_dev[:, None, :, :]  # Shape: (experiments, 1, traces, time samples)

#     num_samples = np.expand_dims(num_trace_arr, axis=(1,2))  # Shape: (total_experiment, 1, 1)

#     # Compute correlation properly (divide by N)
#     num = np.nansum((hypoleak_expanded - precomputed_means)[:, :, :, None] * trace_dev, axis=2) / num_samples
#     den = np.sqrt(precomputed_variances * var_traces) / num_samples

#     r_all = np.abs(num / den)  # Store absolute correlation


#     return r_all
